# 음성인식 오류율 측정 예제

## 환경셋업

In [ ]:
# Run this cell to automatically clone the repo
import sys
import os

if 'google.colab' in sys.modules:
    if not os.path.exists("/content/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" /content/a1003
    %cd /content/a1003/local
else:
    if not os.path.exists("~/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" ~/a1003
    %cd ~/a1003/local

## 개별 문장에 대한 오류율 측정




다음 코드는 두 개의 한국어 문장(정답, 음성인식결과)에 대하여
edit distance(최소편집거리)를 계산하여 WER과 CER을 측정하는 코드입니다.

Edit distance를 계산하기 위해서 `editdistance` 모듈을 import하고,
두 개의 문장을 각 변수에 저장합니다.


In [ ]:
import editdistance

ref_text = "오늘 서울의 날씨가 어때"
hyp_text = "음 오늘의 날씨 가 어때"

WER을 측정하기 위해서 문장을 단어단위로 분리하여 리스트에 저장하빈다.

In [ ]:
ref = ref_text.split()
hyp = hyp_text.split()

print(ref, hyp, sep='\n')

Edit distance를 계산합니다. `editdistance.eval()` 함수는 두개의 리스트를 입력받아서,
두 리스트 간의 editdistance를 반환하는 함수입니다.

In [ ]:
E = editdistance.eval(ref, hyp)
N = len(ref)
WER = E/N*100
print(f"N={N}, E={E}, WER={WER}")

CER을 측정하기 위해서 문장을 각 글자의 리스트로 변환합니다.

In [ ]:
ref = list(ref_text)
hyp = list(hyp_text)
print(ref, hyp, sep='\n')


글자 단위로 분리된 리스트(`ref`, `hyp`)에 대해서 같은 방법으로 editdistance를 측정합니다.
공백문자도 하나의 글자로 처리된 것도 확인하세요.


In [ ]:
E = editdistance.eval(ref, hyp)
N = len(ref)
CER = E/N*100
print(f"N={N}, E={E}, CER={CER}")

이번에는 공백문자를 무시하고(즉, 띄어쓰기 오류를 완전히 무시하고) CER을 측정해 봅니다.
앞에서 글자단위로 분리한 것과 다르게 이번에는 공백문자를 미리 제거합니다.

In [ ]:
ref = list(''.join(ref_text.split()))
hyp = list(''.join(hyp_text.split()))
print(ref, hyp, sep='\n')

In [ ]:
E = editdistance.eval(ref, hyp)
N = len(ref)
CER = E/N*100
print(f"N={N}, E={E}, CER={CER}")

## 여러 문장에 대한 평균 오류율 측정

지금까지는 한개의 {정답,인식결과} 쌍에 대해서 WER, CER을 측정해보았습니다.
실제로는 매우 많은 {정답,인식결과} 쌍에 대해서 오류율을 계산하고 전체의 오류율을 성능지표로 사용합니다.
이를 위해서 `kaldi`와 `ESPnet`은 다음과 같은 형식으로 여러 파일에 대한 정답전사문과 인식결과를 파일로 저장해둡니다.

In [ ]:
!cat data/ref.txt

In [ ]:
!cat data/hyp.txt

이 두개의 파일을 읽어서 utternace-id 가 같은 문장간의 editdistance를 출력하는 파이선 코드가 `uttwer.py`, `uttcer.py` 파일입니다.
다음과 같은 방법으로 실행합니다.

In [ ]:
!python uttwer.py data/ref.txt data/hyp.txt

In [ ]:
!python uttcer.py data/ref.txt data/hyp.txt

다른 데이터에 대해서도 해봅시다.

In [ ]:
!git clone https://hf.co/datasets/pkyoung/a1003.git ./data/a1003
!cp data/a1003/result_espnet_ma16k2401a.txt ./result.txt
!head result.txt



In [ ]:
!python uttcer.py data/a1003/text result.txt

문장부호를 제외한 인식률을 측정해봅니다.

In [ ]:
!python uttwer.py <(python pp.py -s data/a1003/text) <(python pp.py -s result.txt) > /dev/null

화자별 성능도 측정해봅시다.

In [ ]:
%%bash
for s in spk{1..9}; do
  echo -n "$s " >&2
  python uttcer.py data/a1003/text <(grep "$s/" result.txt) > /dev/null
done